In [1]:
import pandas as pd
import numpy as np
import glob

In [4]:
df = pd.read_csv('estat_gov_10a_exp.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8735 entries, 0 to 8734
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   freq     8735 non-null   object
 1   unit     8735 non-null   object
 2   sector   8735 non-null   object
 3   cofog99  8735 non-null   object
 4   na_item  8735 non-null   object
 5   geo      8735 non-null   object
 6   2018     8735 non-null   object
 7   2019     8735 non-null   object
 8   2020     8735 non-null   object
 9   2021     8735 non-null   object
 10  2022     8735 non-null   object
dtypes: object(11)
memory usage: 750.8+ KB


In [5]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t') for file in files}

In [6]:
for col in ["freq", "unit", "sector", "cofog99", "na_item", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,unit,sector,cofog99,na_item,country,2018,2019,2020,2021,2022
0,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Austria,1255.3,1175.0,1046.8,1211.6,1309.4
1,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Belgium,1565.5,1687.1,1761.5,1824.1,2015.8 p
2,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Bulgaria,593.4,748.8,465.5,679.5,763.9
3,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Switzerland,1164.0,1301.4,1418.6,1411.6,1544.4
4,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Cyprus,321.9,346.2,325.6,474.7,551.2


In [7]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [8]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [10]:
df.head()

,freq,unit,sector,cofog99,na_item,country,2018,2019,2020,2021,2022
0,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Austria,1255.0,1175.0,1046.0,1211.0,1309.0
1,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Belgium,1565.0,1687.0,1761.0,1824.0,2015.0
2,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Bulgaria,593.0,748.0,465.0,679.0,763.0
3,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Switzerland,1164.0,1301.0,1418.0,1411.0,1544.0
4,Annual,Million euro,General government,Housing and community amenities,Total general government expenditure,Cyprus,321.0,346.0,325.0,474.0,551.0


In [11]:
for col in df.columns:
    if not col.isdigit() and df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

In [12]:
df.head()

,unit,sector,cofog99,country,2018,2019,2020,2021,2022
0,Million euro,General government,Housing and community amenities,Austria,1255.0,1175.0,1046.0,1211.0,1309.0
1,Million euro,General government,Housing and community amenities,Belgium,1565.0,1687.0,1761.0,1824.0,2015.0
2,Million euro,General government,Housing and community amenities,Bulgaria,593.0,748.0,465.0,679.0,763.0
3,Million euro,General government,Housing and community amenities,Switzerland,1164.0,1301.0,1418.0,1411.0,1544.0
4,Million euro,General government,Housing and community amenities,Cyprus,321.0,346.0,325.0,474.0,551.0


In [13]:
# df.to_csv("./clean/estat_gov_10a_exp.csv", index=False)

In [4]:
df = pd.read_csv('./clean/estat_gov_10a_exp.csv')

In [5]:
cat = ['Health', 'Total']
unit = ['Million euro', 'Percentage of total']
health = df[df.sector == 'General government']
health = health[health.cofog99.isin(cat)]
health = health[health.unit.isin(unit)].sort_values(['country', 'cofog99', 'unit'])
health = health.drop(columns=['sector']).reset_index(drop=True)
health

,unit,cofog99,country,2018,2019,2020,2021,2022
0,Million euro,Health,Austria,31770.0,32971.0,35132.0,40840.0,41735.0
1,Percentage of total,Health,Austria,16.0,17.0,16.0,17.0,17.0
2,Million euro,Total,Austria,187850.0,193308.0,216367.0,227664.0,237757.0
3,Million euro,Health,Belgium,35116.0,36353.0,40151.0,43435.0,44951.0
4,Percentage of total,Health,Belgium,14.0,14.0,14.0,15.0,15.0
...,...,...,...,...,...,...,...,...
94,Percentage of total,Health,Sweden,14.0,14.0,14.0,14.0,14.0
95,Million euro,Total,Sweden,236736.0,236113.0,253531.0,268507.0,269868.0
96,Million euro,Health,Switzerland,13042.0,14085.0,16800.0,19011.0,18018.0
97,Percentage of total,Health,Switzerland,6.0,6.0,6.0,7.0,7.0


In [6]:
health.to_csv('./clean/estat_gov_10a_exp.csv', index=False)